In [1]:
%reset -f
import ROOT
import traceback
import sys
from MyCommonAnalysisFunction_richcap import *
from Convert_MultiDim_Kinematic_Bins  import *
from ExtraAnalysisCodeValues          import *
from array                            import array
from datetime                         import datetime

script_dir = '/w/hallb-scshelf2102/clas12/richcap/SIDIS_Analysis/RC_Correction_Code'
sys.path.append(script_dir)
from Find_RC_Fit_Params import Find_RC_Fit_Params, Apply_RC_Factor_Corrections, Get_RC_Fit_Plot
sys.path.remove(script_dir)
del script_dir

def variable_Title_name_new(variable_in):
    if(variable_in in ["k0_cut"]):
        return "E^{Cutoff}_{#gamma}"
    else:
        output = variable_Title_name(variable_in)
        output = output.replace(" (lepton energy loss fraction)", "")
        return output

ROOT.TH1.AddDirectory(0)
ROOT.gStyle.SetTitleOffset(1.3,'y')
ROOT.gStyle.SetGridColor(17)
ROOT.gStyle.SetPadGridX(1)
ROOT.gStyle.SetPadGridY(1)
ROOT.gStyle.SetStatX(0.80)  # Set the right edge of the stat box (NDC)
ROOT.gStyle.SetStatY(0.45)  # Set the top edge of the stat box (NDC)
ROOT.gStyle.SetStatW(0.3)   # Set the width of the stat box (NDC)
ROOT.gStyle.SetStatH(0.2)   # Set the height of the stat box (NDC)


print(f"{color.BOLD}\nStarting RG-A SIDIS Analysis\n{color.END}")

Relative_Background_Run_Q = False



Starting RG-A SIDIS Analysis



In [2]:
TTree_Name = "Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root"
print(f"\n{color.BBLUE}Using Existing TTree {color.END_B}(File is: {color.UNDERLINE}{TTree_Name}{color.END_B}){color.END}\n")
TTree_Input = ROOT.TFile.Open(TTree_Name, "READ")
# List_of_All_Histos_For_Unfolding = {}
count = 0
length_of_name = 0
for key in TTree_Input.GetListOfKeys():
    key_name = key.GetName()
    if(not Relative_Background_Run_Q):
        Relative_Background_Run_Q = "Relative_Background" in str(key_name)
    if("TList_of_" in key_name):
        count += len([float(str(item.GetString())) for item in TTree_Input.Get(key_name)])
        pass
        # List_of_All_Histos_For_Unfolding[key_name] = [float(str(item.GetString())) for item in TTree_Input.Get(key_name)]
    else:
        print(f"\t{key_name:>112}-> ")
        length_of_name = max([length_of_name, len(str(key_name))])
        count += 1
        # List_of_All_Histos_For_Unfolding[key_name] = TTree_Input.Get(key_name)
TTree_Input.Close()
print(f"\n{color.BBLUE}Recovered: {color.BGREEN}{count}{color.END_B}{color.BLUE} items{color.END}")
print(f"\n{color.BOLD}length_of_name = {length_of_name}{color.END}\n")


Using Existing TTree (File is: Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root)


Recovered: 52754 items

length_of_name = 110

